In [3]:
import numpy as np
import tensorflow as tf

from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional

from constants import *
from encoding_decoding_lookup import *

## Training our model

### Dataset

### Diacritizer

In [ ]:
# for training we need to follow these steps:
'''
1. input is from lookup table all sentences concatenated together
2. For variable length input we need sliding window and also it helps to get the contextual meaning due to overlapping
'''

In [ ]:
# # just like the lab, but with tf, we would do init with all needed variables and a forward fn
# #what we need: self, window_size, lstm_size, dropout_rate,embedding_size
# class Diacritizer:
#     def __init__(self)
#     def __init__(self, vocab_size=35181, embedding_dim=50, hidden_size=50, n_classes=len(tag_map)):
    

In [ ]:
# not sure if we need vocab size since we use window
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional


class Diacritizer(Model):
    def __init__(self, embedding_size=DEFAULT_EMBEDDING_SIZE, lstm_size=DEFAULT_LSTM_SIZE, window_size=DEFAULT_WINDOW_SIZE,dropout_rate=DEFAULT_DROPOUT_RATE):
        super(Diacritizer, self).__init__()

        ######################################## Creating the layers of our model #####################

        # Step 1: define the input
        # input would be the fixed window size after lookup 
        # Input in tf defines the input layer with shape=(window_size,)
        # it represents the input data.
        self.inputs = Input(shape=(window_size,), name='input')


        # Step 2: define the embeddings
        # it converts the integer index to dense vectors with fixed size
        # input dim is dim of the letters (length of valid input letters we have)
        # the output is the dense vector (embeddings size which is 128)
        # the ( ) at the end mean that this layer would be applied to the input layer
        self.embedding = Embedding(input_dim= len(SORTED_VALID_INPUT_LETTERS) + 1, output_dim= embedding_size, name='embedding')(self.inputs)
        

        # Step 3: Define the Bidirectional LSTM layers (we have 4 layers of each class and an initial layer)
        
        # initial layer that would be applied to the embeddings
        self.initial_layer = Bidirectional(LSTM(lstm_size, dropout=dropout_rate, return_sequences=True),
                                      name='initial_layer')(self.embedding)
        
        # first layer is sukun layer that would be applied on initial layer
        self.sukoon_layer = Bidirectional(LSTM(lstm_size, dropout=dropout_rate, return_sequences=True),
                                     name='sukoon_layer')(self.initial_layer)
        
        # sec layer is shadda layer that would be applied to sukun layer
        self.shadda_layer = Bidirectional(LSTM(lstm_size, dropout=dropout_rate, return_sequences=True),
                                     name='shadda_layer')(self.sukoon_layer)
        
        # third layer is sec layer that would be applied to shadda layer
        self.secondary_diacritics_layer = Bidirectional(LSTM(lstm_size, dropout=dropout_rate, return_sequences=True),
                                                   name='secondary_diacritics_layer')(self.shadda_layer)
        
        # forth layer is primary layer that would be applied to sec layer
        self.primary_diacritics_layer = Bidirectional(LSTM(lstm_size, dropout=dropout_rate, return_sequences=True),
                                                 name='primary_diacritics_layer')(self.secondary_diacritics_layer)

        # Step 4: Define the output layers (we have 4 outputs with diff dense)
        # dense functions define the activation functions in tensor flow
        # where dense=1 means sigmoid and dense =2 means softmax

        # Sukun output would have a sigmoid as its binary and would come from the sukun layer
        # dense= 1 means single neuorn we just wanna know if 0 no sukun or 1 yes sukun
        # SIGMOID
        self.sukoon_output = Dense(1, name='sukoon_output')(self.sukoon_layer)

        # shadda output
        # SIGMOID
        self.shadda_output = Dense(1, name='shadda_output')(self.shadda_layer)

        # sec output
        # dense= 4 as we have multi classification task, where each neuron assigned to one class
        # we have 0,1,2,3 as per the lookup table
        # SOFTMAX
        self.secondary_diacritics_output = Dense(4, name='secondary_diacritics_output')(self.secondary_diacritics_layer)

        # prim output
        # SOFTMAX
        self.primary_diacritics_output = Dense(4, name='primary_diacritics_output')(self.primary_diacritics_layer)
       

    def forward(self, inputs):
        # Step 4: Define the forward pass through the layers
       

        # Step 5: Separate outputs for each diacritic
       

        return primary_output, secondary_output, shadda_output, sukoon_output


--- 